## Removing non-wind rows/cols of $Q_{obj}$



In [45]:
function tmp_inst_Qobj(n,nr,T)
    """ Generate the objective function matrix
    Qobj from the problem dimensions.
    Assume no correlation between wind sites.
    """
    return sparse(diagm(repeat([ones(nr),zeros(n)],outer=[T])))
end

tmp_inst_Qobj (generic function with 1 method)

In [46]:
n = 4
nr = 1
T = 2

Qobj = tmp_inst_Qobj(n,nr,T)
full(Qobj)

10x10 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

## Removing non-wind columns of $A$

Only a small subset of the nodes in a transmission network will have wind farms. The previous development of $A$ assumed wind farms at all nodes and resulted in a matrix with $2nT$ columns. There are, however, only $(n+n_R T)$ variables, where $n_R$ is the number of wind farms. This may be broken down into $(n-1)$ angle variables, $1$ mismatch variable, and $n_R$ wind deviation variables per time step.

The function that generates $A$ must accept an additional argument: a vector indicating which ndoes have wind farms.

In [12]:
function tmp_inst_A(n,Ridx,T,Y,k)
    """ Generate the power balance constraint A matrix
    from problem dimensions, admittance matrix,
    and generator participation factors.
    Assumes the admittance matrix is (n-1)-by-(n-1).
    """
    # A = zeros((n+1)*T,2*n*T)
    
    # A has a block diagonal pattern where each
    # block is Atemp:
    Atemp = zeros((n+1),2*n)
    Atemp[1:n,1:n] = -eye(n)
    Atemp[end,1:n] = ones(1,n)
    Atemp[1:n-1,n+1:2n-1] = Y
    Atemp[:,2*n] = [-k,1]
    
    # Remove columns corresponding to non-wind nodes:
    Atemp = sparse(Atemp[:,[Ridx,n+1:2*n]])
    
    # Now we can tile the Atemp matrix to generate A:
    A = Atemp
    for t = 2:T
        A = blkdiag(A, Atemp)
    end
    
    return A
end

tmp_inst_A (generic function with 1 method)

In [15]:
n = 4
T = 2
Ridx = [1,3]
length(Ridx)
Y = eye(3)
k = [0.3,0.3,0.3,0.1]

A = tmp_inst_A(n,Ridx,T,Y,k)
full(A)

10x12 Array{Float64,2}:
 -1.0   0.0  1.0  0.0  0.0  -0.3   0.0   0.0  0.0  0.0  0.0   0.0
  0.0   0.0  0.0  1.0  0.0  -0.3   0.0   0.0  0.0  0.0  0.0   0.0
  0.0  -1.0  0.0  0.0  1.0  -0.3   0.0   0.0  0.0  0.0  0.0   0.0
  0.0   0.0  0.0  0.0  0.0  -0.1   0.0   0.0  0.0  0.0  0.0   0.0
  1.0   1.0  0.0  0.0  0.0   1.0   0.0   0.0  0.0  0.0  0.0   0.0
  0.0   0.0  0.0  0.0  0.0   0.0  -1.0   0.0  1.0  0.0  0.0  -0.3
  0.0   0.0  0.0  0.0  0.0   0.0   0.0   0.0  0.0  1.0  0.0  -0.3
  0.0   0.0  0.0  0.0  0.0   0.0   0.0  -1.0  0.0  0.0  1.0  -0.3
  0.0   0.0  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0  0.0  -0.1
  0.0   0.0  0.0  0.0  0.0   0.0   1.0   1.0  0.0  0.0  0.0   1.0

## Removing non-wind parts of $Q_\theta$



In [49]:
function tmp_inst_Qtheta(n,nr,T,tau,line)
    """ Generate Q_theta in the temperature constraint
    of a temporal instanton problem instance.
    "line" has the form (i,k), where i and k refer to
    the endpoints of the chosen line.
    """
    Qtheta = zeros((n+nr)*T,(n+nr)*T)
    i,k = line
    ei = zeros(n-1,1)
    ei[i] = 1
    ek = zeros(n-1,1)
    ek[k] = 1
    
    Q0 = (ei - ek)*(ei - ek)'
    
    for t = 1:T
        start = nr + 1 + (nr+n)*(t-1)
        stop = start + n - 2
        Qtheta[start:stop,start:stop] = tau^(T-t)*Q0
    end
    return sparse(Qtheta)
end

tmp_inst_Qtheta (generic function with 2 methods)

In [53]:
# Test generateA and generateb:
n = 4
nr = 2
T = 2
tau = 2
line = (1,3)
Y = [2 3; 3 4]
k = [0.25,0.25,0.5]

Qtheta = tmp_inst_Qtheta(n,nr,T,tau,line)
full(Qtheta)

12x12 Array{Float64,2}:
 0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0
 0.0  0.0   2.0  0.0  -2.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0
 0.0  0.0  -2.0  0.0   2.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0   1.0  0.0  -1.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  -1.0  0.0   1.0  0.0
 0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0

In [48]:
# Test generateA and generateb:
n = 3
T = 2
tau = 2
line = (1,2)
Y = [2 3; 3 4]
k = [0.25,0.25,0.5]

Qtheta = tmp_inst_Qtheta(n,T,tau,line)

12x12 sparse matrix with 8 Float64 entries:
	[4 ,  4]  =  2.0
	[5 ,  4]  =  -2.0
	[4 ,  5]  =  -2.0
	[5 ,  5]  =  2.0
	[10, 10]  =  1.0
	[11, 10]  =  -1.0
	[10, 11]  =  -1.0
	[11, 11]  =  1.0

# Connecting to solver machinery

Previously I wrote a few methods to solve the temporal instanton problem according to the procedure described by Dan Bienstock (see the "Trust Region Subproblem" notebook). Now I will connect my most recent work in modeling the problem to this solution method.

I believe the trust region subproblem code should be in its own module. It is, after all, wholly independent of the instanton application.

My trust region subproblem code assumed a problem of the form

\begin{align}
&& \min & G(x) \\
&& s.t.  & \\
&&  Ax &= b \\
&& A_{i\cdot}x &\geq 0 \quad i\in \mathcal{R} \\
&&  x^\top Qx &= c
\end{align}